# Classification of Tree Species with Machine Learning

$\qquad\qquad\qquad\qquad\qquad\qquad\qquad\qquad$<img src="https://drive.google.com/uc?id=1F4B80Q7XFcMK9aDKXGYRKgKw7q8sRP5i" alt="Cache la Poudre"  width="300"/>
<center>Cache la Poudre Natural Reserve</center>

We will work in this practical on classifying tree species using topographic features, with the Cache La Poudre Natural Reserve dataset provided by *Blackard, Jock A. 1998. "Comparison of Neural Networks and Discriminant Analysis in Predicting Forest Cover Types." Ph.D. dissertation. Department of Forest Sciences. Colorado State University. Fort Collins, Colorado.*


The objective of this exercise is to test a series of machine learning algorithms for the classification of tree species. The dataset is comprised of $30\times30$m parcels, which we want to classify among the following species:

> **Class Index** | **Class Name**
> --- | ---
> 0 | Spruce/fir
> 1  | Lodgepole pine
> 2  | Ponderosa pine
> 3  | Cottonwood/willow
> 4  | Aspen poplar
> 5  | Douglas-fir
> 6  | Krummholz

For each parcels, we have 10 handcrafted descriptors designed by experts. they are as follow:


> **Index** | **Unit** | Description
> --- | --- | ---
> 0 | meter | Elevation
> 1 | degree | Azimuth
> 2 | degree | Slope
> 3 | meter | Distance (horizontal) to nearest water body
> 4 | meter | Distance (vertical) to nearest water body
> 5 | meter | Distance (horizontal) to nearest road
> 6 | index (0-255) | Shadow at 9am
> 7 | index (0-255) | Shadow at 12am
> 8 | index (0-255) | Shadow at 3pm
> 9 | meter | Distance (horizontal) to nearest known fire outbreak


All the code that needs to be completed is marked down with `#TODO`. Make sure you complete them all before moving on to the next question. Do not hesitate to create new cells and copy some code to play around with the tensors and check their size, it can be difficult to debug a notebook otherwise. You will be required to look up some documentations on numpy and sklearn online.

We have added some `assert` after some cells, make sure that they pass without error before moving to the next question.

## Installation and Data Download

**Q1**
Execute the cell [1] and [2], click the authentication link, copy the key and paste it in the box. Execute the data loader cell [3]



In [ ]:
#[1] import and installations
!pip install ucimlrepo
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from ucimlrepo import fetch_ucirepo
import pandas as pd

In [ ]:
#[3] building the train and test sets
data_file = fetch_ucirepo(id=31)
features_names = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']
data = np.array(data_file['data']['features'][features_names])


In [ ]:
#print(data_file['data']['targets'].columns)
label = np.array(data_file['data']['targets']['Cover_Type'][:]).squeeze()
class_names = ["Spruce/Fir", "Lodgepole Pine", "Ponderosa Pine", "Cottonwood/Willow", "Aspen Poplar", "Douglas-Fir", "Krummholz"]

# work only on a sample of the data
perm = np.random.permutation(data.shape[0])
data = data[perm[0:10000],:]
label = label[perm[0:10000]]-1

**Q2** Execute cell [4]. What can we say about the species repartition? What is the risk for the classification results if we don't remedy this issue?

In [ ]:
#[4]
bincount = plt.hist(label, bins=[0,1,2,3,4,5,6])
print('\n'.join('{:20s} : {:5.0f}'.format(name, binc) for name, binc in zip(class_names,bincount[0])))

**Q3** Complete cell [5] to normalize the input values to have zero mean and std one. Why is it not necessary to scale the last 4 features?

In [ ]:
print(data.shape)

(10000, 10)


In [ ]:
#[5]
def scale_data(x):
  return (x - #TODO)/#TODO
data = np.apply_along_axis(scale_data,0,data)
assert((np.abs(np.mean(data,0))<1e-3).all())
assert((np.abs(data[:,0:10].std(0))-1<1e-4).all())

## Loss Function and Metrics

We consider the two following metrics, defined with respect to the confusion matrix $CM$ of size $k \times k$:

**Overall Accuracy.** A global metric defined as the ratio of correct prediction divided by the number of (annotated) points
    $$
    OA = \frac{\text{Trace}(CM)}
    {\text{Sum}(CM)}.
    $$
**Class IoU.** This per-class metric is defined as the ratio between true positives divided by the sum of false positives, false negatives and true positives.
    $$
    IoU_i = \frac{CM_{i,i}}
    {CM_{i,i} + \sum_{j \neq i}\left(CM_{i,j} + CM_{j,i} \right)}
    .
    $$

**Q4** Complete cell [6] for the computation of OA and IoU.
Run the cell [7] and make sure you obtain the following:

`OA = 71.43% `

`Spruce/fir : 60.00% `

`lodgepole pine : 25.00% `

`Ponderosa pine : 100.00% `

`Cottonwood/willow : 100.00% `

`Aspen poplar : 100.00% `

`Douglas-fir : 66.67%`

`Krummholz : 33.33%`

In [ ]:
#[6]
class ConfusionMatrix:
  def __init__(self, n_class, class_names):
    self.CM = np.zeros((n_class, n_class))
    self.n_class = n_class
    self.class_names = class_names

  def clear(self): #reset the CM to 0
    self.CM = np.zeros((self.n_class, self.n_class))

  def add(self, gt, pred): #add a set of prediction pred with respective gt
    """
    gt : vector of size N
    pred: vector of size N
    """
    self.CM +=  confusion_matrix(gt, pred, labels = list(range(self.n_class)))

  def overall_accuracy(self):#percentage of correct classification
    return 100*#TODO use HINT: np.trace

  def class_IoU(self, show = 1):
    ious = np.full(self.n_class, 0.)
    for i_class in range(self.n_class): #this actually can be done without a loop!
      ious[i_class] = #TODO HINT: use np.diag,  np.sum()
    if show:
      print(' | '.join('{} : {:3.2f}%'.format(name, 100*iou) for name, iou in zip(self.class_names,ious)))
    #do not count classes that are not present in the dataset in the mean IoU
    return 100*np.nansum(ious) / (np.logical_not(np.isnan(ious))).sum()

In [ ]:
#[7]
m = ConfusionMatrix(7, class_names)
m.add(np.array([0,1,1,5,2,0,0,4,0,5,3,6,5,1]), np.array([0,1,0,5,2,0,1,4,0,5,3,6,6,6]))
print(m.CM)
print("OA = %3.2f%%" % (m.overall_accuracy()))
m.class_IoU()
m.clear()

**Q5** Execute cell [8] to perform the train/test split. Explain briefly the role of these two datasets.

In [ ]:
#[8]
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=0)

## SVM Classifier

**Q6** Execute cell [9] to train and evaluate a linear SVM. Read the online doc and try the following kernels: `'rbf', 'poly', 'sigmoid'`. Which one perform better? Set `class_weight=None`, and comment the effect on the mIoU and the OA.

In [ ]:
#[9]
svm_classifier = svm.SVC(kernel='linear', C=100, class_weight='balanced', gamma = 'scale').fit(X_train, y_train)
pred_svm = svm_classifier.predict(X_test)
m.clear()
m.add(y_test, pred_svm)
print("OA = %3.2f%%" % (m.overall_accuracy()))
print("mIoU = %3.2f%%" % (m.class_IoU()))

**Q7** Complete cell [10] to evaluate the performance of all polynomial kernel degree 1 to 10. What is the optimal kernel size? Why is the performance dropping if the degree is too small or too large?

In [ ]:
#[10]
deg_array = list(range(1,10)) #list of all degrees we will try
n_try = len(deg_array)
mIoU_array = np.zeros((n_try,)) #array storing the miou for each configuration
for i in range(n_try):
  svm_classifier = svm.SVC(kernel='poly', degree=#TODO #set the SVM parameters
  pred_svm = #TODO #prediction with SVN above
  m.clear()
  m.add(#TODO #put the prediction into a confusion matrix
  mIoU_array[i] = #TODO #store the miou in the array
  print("%i / %i : deg = %4.2f -> mIoU = %2.1f" % (i, n_try-1, deg_array[i], mIoU_array[i]))
plt.plot(deg_array, mIoU_array)

**Q8** Complete cell [11] to find the optimal value of $C$ for a polynomial kernel of the degree of 3.
Run your best model in cell [12]

In [ ]:
#[11]
C_array = [1,5,10,20,50,200,500,1000,2000,5000]
#TODO
for i in range(n_try):
  svm_classifier = #TODO
  #TODO: fill mIoU_array
  print("%i / %i : C = %4.2f -> mIoU = %2.1f" % (i, n_try-1, C_array[i], mIoU_array[i]))
plt.plot(C_array, mIoU_array)

In [ ]:
#[12]
svm_classifier = svm.SVC(#TODO - best model
pred_svm = #TODO
m.clear()
m.add(#TODO
print("OA = %3.2f%%" % (m.overall_accuracy()))
print("mIoU = %3.2f%%" % (m.class_IoU()))

## KNN and RandomForest Classifier

**Q9** In cell [13], perform the same analysis for the KNN classifier and find the optimal number of neighbors.

In [ ]:
#[13]
knn_classifier = KNeighborsClassifier(n_neighbors=10).fit(X_train, y_train)

**Q10** In cell [14], perform the same analysis for the Random Forest classifier and find the optimal `max_depth`

In [ ]:
#[14]
rf_classifier = RandomForestClassifier(n_estimators=500, max_depth=2,random_state=0,class_weight='balanced').fit(X_train, y_train)

$\star$ **Q11** Evaluate the influence of the number of trees in the forest (`n_estimators`), and comment on the effect of performance and runtime.

$\star$ **Q12** Evaluate the importance of all inputs with your best performing forest (use the online doc). Perform the classification using the best 5 inputs and comment on the results.

$\star$ **Q13** In cell [16], implement a 5 fold cross validation on the dataset using `sklearn.model_selection.cross_val_score` for your best performing model.

$\star\star$ **Q14** Complete the function `consensus` in cell [15] which takes a list of prediction and perform a majority vote. Try a consensus voting between your best implementation of knn, SVM, and RF: the class predicted is the class with the most "votes". in case of equality, chose the best model. Comment on the results

In [ ]:
#[15]
def consensus(pred_list, n_class = 7):
  """ takes a list of prediction (vectors of size N) and perform a consensus vote
  outputs a vector of size N with the prediction from consensus
  """
  n_points = pred_list[0].shape[0]
  pred_vote = np.zeros((n_points, n_class)) #store the votes
  for pred in pred_list:
    pred_vote[list(range(n_points)), #TODO] += 1 #count the votes for each element of the dataset
  consensus_vote = #use argmax
  return consensus_vote

In [ ]:
#[16]
pred_svm = #TODO - on test set
pred_knn = #TODO - on test set
pred_forest = #TODO - on test set
#TODO
pred_consensus = consensus([#TODO])
m.clear()
m.add(y_test, pred_consensus)
print("OA = %3.2f%%" % (m.overall_accuracy()))
print("mIoU = %3.2f%%" % (m.class_IoU()))